Modèle Stat ARIMA... (SARIMA)

librairie : statmodel (library, package ARIMA)


Model LLM, Ollama, TimeCHATGPT...

à compléter et battre le BTC tout seul est quasiment impossible 

In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, BatchNormalization
from tensorflow.keras.activations import linear, relu, sigmoid
from modules.utils import features_engineering

from sklearn.model_selection import train_test_split

In [2]:
X = pd.read_csv('/Users/elouan/Repo Github ElouanBahri/Predicting_crypto_prices/Historical Prices for BTCUSDT')


In [3]:
# YEARS = [2019,2021,2022,2023,2024,2025]

# Data = filter_data_by_year_month(X, YEARS)

df = features_engineering(X)



In [8]:
# Check for NaN values in the entire DataFrame
print(df.isnull().sum())


open                            0
high                            0
low                             0
close                           0
volume                          0
quote_asset_volume              0
number_of_trades                0
taker_buy_base_asset_volume     0
taker_buy_quote_asset_volume    0
is_weekend                      0
hour_sin                        0
hour_cos                        0
day_of_week_sin                 0
day_of_week_cos                 0
year_sin                        0
year_cos                        0
price_change                    0
price_change_pct                0
high_low_range                  0
volume_change_pct               0
taker_buy_ratio                 0
ma_5                            0
ma_10                           0
rsi                             0
bb_upper                        0
bb_lower                        0
close_lag_1                     0
volume_lag_1                    0
close_lag_2                     0
volume_lag_2  

In [ ]:
df

In [14]:
df = df.replace([np.inf, -np.inf], np.nan)  # Replace inf with NaN
df = df.fillna(df.mean())

X = df.drop(columns=['target']).values  # Features
y = df['target'].values  # Target variable


# Reshape the data into sequences (timesteps)
timesteps = 10  # Number of timesteps for the RNN
X_sequences = []
y_sequences = []

for i in range(len(X) - timesteps):
    X_sequences.append(X[i:i+timesteps])
    y_sequences.append(y[i+timesteps])

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)


In [5]:
X_train[0][0]

array([-7.87709367e-01, -7.88899518e-01, -7.89730557e-01, -7.89762862e-01,
       -9.99217709e-02, -4.30374220e-01, -4.08140666e-01, -1.47285037e-01,
       -4.42131781e-01,  0.00000000e+00, -5.00000000e-01,  8.66025404e-01,
        7.81831482e-01,  6.23489802e-01,  1.00000000e+00,  6.12323400e-17,
       -3.86208127e-01, -4.88964445e-01, -3.17716021e-01, -2.10802381e-01,
        4.52675233e-01, -7.87217515e-01, -7.86324614e-01, -2.50848889e-01,
       -7.84796127e-01, -7.87800747e-01, -7.87708540e-01,  4.29256077e-02,
       -7.85717869e-01, -1.71345793e-01, -7.86271547e-01,  1.91374078e-01])

In [16]:
print(np.isnan(X).any(), np.isnan(y).any())  # Check for NaN
print(np.isinf(X).any(), np.isinf(y).any())

False False
False False


In [ ]:
# dataset = tf.data.Dataset.from_tensor_slices((X1, y))
# # Calculate sizes for train and validation sets
# dataset_size = len(X1)  # Total number of samples in your dataset
# train_size = int(0.8 * dataset_size)

# # Shuffle the dataset if needed
# dataset = dataset.shuffle(buffer_size=dataset_size)

# # Split the dataset
# train_dataset = dataset.take(train_size)       # Take the first 80%
# val_dataset = dataset.skip(train_size)         # Skip the first 80%, take the remaining 20%

# # Batch both datasets if needed
# batch_size = 32
# train_dataset = train_dataset.batch(batch_size)
# val_dataset = val_dataset.batch(batch_size)

In [19]:
tf.random.set_seed(1234) # for consistent results

# Simplified GRU Model
model = Sequential([
    # Single GRU Layer
    GRU(64, input_shape=(10, X_sequences.shape[2]), name="GRU_Layer"),
    Dropout(0.2, name="Dropout_Layer"),  # Regularization

    # Fully Connected Dense Layer
    Dense(32, activation='relu', name="Dense_Layer"),  # Hidden dense layer

    # Output Layer for Binary Classification
    Dense(1, activation='sigmoid', name="Output_Layer")  # Predict binary output
])

/Users/elouan/anaconda3/envs/Predicting_crypto/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.summary()

In [ ]:
model = tf.keras.models.load_model("../models/")

In [22]:

# Compile the model
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),  # Binary classification loss
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),      # Adam optimizer
    metrics=['accuracy']                                         # Track accuracy
)

# Learning rate reduction callback
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Monitor validation loss
    factor=0.5,          # Reduce learning rate by a factor of 0.5
    patience=3,          # Wait 3 epochs with no improvement before reducing
    min_lr=1e-6          # Set a minimum learning rate
)

# Model checkpoint callback
checkpoint = ModelCheckpoint(
    filepath='../models/RNN_model_2.keras',  # Save the model to this path
    monitor='val_accuracy',                 # Monitor validation accuracy
    mode='max',                             # Save when accuracy is maximized
    save_best_only=True,                    # Save only if the model improves
    verbose=1                               # Display a message when saving
)

# Train the model
history = model.fit(
    X_train, y_train,                         # Training data
    validation_data=(X_test, y_test),         # Validation data
    epochs=50,                                # Train for 20 epochs
    batch_size=32,                            # Batch size
    callbacks=[reduce_lr, checkpoint]         # Use the callbacks
)


Epoch 1/50
6501/6502 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5483 - loss: 0.6854
Epoch 1: val_accuracy improved from -inf to 0.52215, saving model to ../models/RNN_model_2.keras
6502/6502 ━━━━━━━━━━━━━━━━━━━━ 31s 4ms/step - accuracy: 0.5483 - loss: 0.6854 - val_accuracy: 0.5221 - val_loss: 0.6946 - learning_rate: 0.0010
Epoch 2/50
6500/6502 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5458 - loss: 0.6860
Epoch 2: val_accuracy did not improve from 0.52215
6502/6502 ━━━━━━━━━━━━━━━━━━━━ 28s 4ms/step - accuracy: 0.5458 - loss: 0.6860 - val_accuracy: 0.5201 - val_loss: 0.6950 - learning_rate: 0.0010
Epoch 3/50
6498/6502 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5472 - loss: 0.6853
Epoch 3: val_accuracy did not improve from 0.52215
6502/6502 ━━━━━━━━━━━━━━━━━━━━ 28s 4ms/step - accuracy: 0.5472 - loss: 0.6853 - val_accuracy: 0.5208 - val_loss: 0.6956 - learning_rate: 0.0010
Epoch 4/50
6501/6502 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5484 - loss: 0.6851
Epoch 4: val_accu

KeyboardInterrupt: 

In [23]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train.reshape(X_train.shape[0], -1), y_train)
print(xgb.score(X_test.reshape(X_test.shape[0], -1), y_test))


0.5170351855412421
